# Model 2: The Production Mix Problem I

$$
\begin{array}{rl}
    \max  & 1.6\, x_1 + x_2 + 2\, x_3 \\[5pt]
    \text{s.t.} & 80\,x_1 + 70\,x_2 + 30\,x_3 \leq 300,\!000 \\
    & 70\,x_1 + 90\,x_2 + 20\,x_3 \leq 250,\!000 \\
    &40\,x_1 + 30\,x_2 + 20\,x_3 \leq 180,\!000 \\[5pt]
    & x_1,x_2,x_3 \geq 0 
\end{array}
$$

## The concrete Pyomo model

In this section, we implement the above concrete model step by step.

### Step 1: Import required libraries

In [ ]:
!pip install gurobipy pyomo

In [ ]:
solver_options = {
    "WLSACCESSID": "...",  # your WSL access id (string)
    "WLSSECRET": "...",  # your WSL secret (string)
    "LICENSEID": ...,  # your license id (integer)
}

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

### Step 2: Create the model object

In [ ]:
mod = pyo.ConcreteModel(name="production_mix_1")

### Step 3: Define the decision variables and their domains

In [ ]:
mod.x1 = pyo.Var(name="x1", domain=...)
mod.x2 = pyo.Var(name="x2", domain=...)
mod.x3 = pyo.Var(name="x3", domain=...)

### Step 4: Define the objective function

In [ ]:
mod.obj = pyo.Objective(expr=1.6 * mod.x1 + mod.x2 + 2 * mod.x3, sense=...)

### Step 5: Define the constraints

In [ ]:
mod.con1 = pyo.Constraint(expr=80 * mod.x1 + 70 * mod.x2 + 30 * mod.x3 <= 300000, name="assembly")
mod.con2 = pyo.Constraint(expr=70 * mod.x1 + 90 * mod.x2 + 20 * mod.x3 <= 250000, name="refinement")
mod.con3 = pyo.Constraint(
    expr=40 * mod.x1 + 30 * mod.x2 + 20 * mod.x3 <= 180000, name="quality_control"
)

### Step 6: Create the Solver object and solve the model

In [ ]:
opt = SolverFactory("gurobi", solver_io="python", manage_env=True, options=solver_options)
result = opt.solve(mod)

### Step 8: Display and interpret the results

In [ ]:
print("Objective value =", mod.obj())  # using the object itself
print("x1 =             ", mod.x1())
print("x2 =             ", mod.x2())
print("x3 =             ", pyo.value(mod.x3))  # using the pyo.value() function

Display the solution time:

In [ ]:
print("solution time =", result.solver.wallclock_time)  # for gurobi
# print("solution time =", result.solver.time)  # for glpk

#### Find the slack value of each constraint:

$$
\begin{array}{lcl}
    80\,x_1 + 70\,x_2 + 30\,x_3 \leq 300,\!000 & \Rightarrow & 80\,x_1 + 70\,x_2 + 30\,x_3 + s_1 = 300,\!000 \\
    70\,x_1 + 90\,x_2 + 20\,x_3 \leq 250,\!000 & \Rightarrow & 70\,x_1 + 90\,x_2 + 20\,x_3 + s_2 = 250,\!000 \\
    40\,x_1 + 30\,x_2 + 20\,x_3 \leq 180,\!000 & \Rightarrow &40\,x_1 + 30\,x_2 + 20\,x_3 + s_3 = 180,\!000 \\
\end{array}
$$

> *Note*: In an optimization problem, slack and surplus variables are variables that are added to the inequality constraints to transform them into equality. Slack refers to the amount which is equal to or less than ($\leq$) constraints, while surplus refers to the amount which is equal to or greater than ($\geq$) constraints. Both values will be equal to 0 if a constraint is perfectly met as an equivalence. If a slack or surplus variable is positive at a particular candidate solution, the constraint is *non-binding* there, as the constraint *does not restrict* the possible changes from that point.

In [ ]:
# by using the slack() method:
print("Slack value of", mod.con1.name, "=", mod.con1.slack())
print("Slack value of", mod.con2.name, "=", mod.con2.slack())

# by using the upper() and body() methods and performing calculations:
print("Slack value of", mod.con3.name, "=", mod.con3.upper() - mod.con3.body())